# Laboratorium 1 - content-based recommender

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn`

## Część 1. - przygotowanie danych

In [140]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split, KFold
from scipy.special import comb 

import sklearn.metrics 

np.random.seed(123)

In [141]:
# tworzymy reprezentacje filmow jako wektorow cech - na podstawie gatunkow

genres = [
    '(no genres listed)', 
    'Action', 
    'Adventure', 
    'Animation', 
    'Children', 
    'Comedy', 
    'Crime', 
    'Documentary', 
    'Drama', 
    'Fantasy', 
    'Film-Noir', 
    'Horror', 
    'IMAX', 
    'Musical', 
    'Mystery', 
    'Romance', 
    'Sci-Fi', 
    'Thriller', 
    'War', 
    'Western'
]
genres_no = len(genres)

movies = pandas.read_csv('ml-latest-small/movies.csv')
movies_no = movies.shape[0]

movies['bias'] = 1.0
for genre in genres:
    movies[genre] = np.where(movies['genres'].str.contains(genre, regex=False), 1.0, 0.0)
    
movies = movies.drop(columns=['title', 'genres']).set_index('movieId')
movies

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
# wczytujemy oceny uytkownikow i od razu dzielimy je na dwa zbiory - treningowy i testowy

all_ratings = pandas.read_csv('data/ml-latest-small/ratings.csv').drop(columns=['timestamp'])
train_ratings_set, test_ratings_set = train_test_split(all_ratings, test_size=0.05)
train_ratings_set

,userId,movieId,rating
43640,292,2989,4.0
39095,270,784,1.0
34478,232,43904,2.0
40148,274,7845,3.5
25125,177,1953,2.5
...,...,...,...
63206,414,2707,3.0
61404,404,515,3.0
17730,111,102407,3.5
28030,192,590,4.0


In [143]:
# inicjalizujemy macierz preferencji uzytkownikow liczbami losowymi z przedzialu [0.0, 5.0]

def initialize_users(raw_ratings):
    users_no = raw_ratings['userId'].unique().size
    users = pandas.DataFrame(5.0 * np.random.uniform(size=(users_no, genres_no+1)), index=raw_ratings['userId'].unique(), columns=['bias']+genres)
    return users_no, users

users_no, users = initialize_users(train_ratings_set)
users

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
292,1.014690,0.995819,1.528380,0.257574,0.824470,0.087500,3.980853,2.062487,3.114433,4.405692,...,0.993665,0.852595,0.784303,1.218735,0.515662,4.485269,4.729992,2.435687,3.477210,3.939376
270,4.308478,2.879758,1.810808,1.092392,4.109795,1.204434,4.695290,1.059030,1.974731,3.107042,...,4.125993,1.152395,1.003104,3.370786,2.999765,3.065997,0.054319,4.415214,0.709349,4.507762
232,3.067525,1.748734,2.303163,4.704785,1.601667,0.072212,4.122613,3.353317,3.692728,2.416649,...,1.551154,0.804891,4.764028,2.041699,3.284680,3.379268,4.523883,4.069649,1.418443,1.950233
274,0.833575,4.653115,3.227781,2.709112,0.683983,3.115202,3.358098,3.517350,0.083922,4.222237,...,2.163962,1.216554,1.675145,4.453842,4.851751,4.628583,0.274855,4.686171,0.277883,4.199957
177,4.302558,0.861957,1.137921,1.519534,3.947084,3.956545,4.185998,0.521279,4.801370,3.935995,...,1.139384,0.976215,2.297242,0.003728,4.813174,2.490864,2.696603,1.013333,2.861124,2.606113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,1.791684,3.417988,3.120571,4.772421,4.729993,0.122827,2.289694,0.470059,1.294628,2.896644,...,0.069832,2.713075,2.759041,3.299633,0.383510,3.228749,4.390937,2.745361,1.777799,4.015662
473,2.328592,1.729846,0.862862,2.401691,4.672526,2.023391,0.103821,0.753568,0.876652,1.356589,...,2.324049,1.297873,2.473906,2.634229,4.012611,0.014786,4.609693,3.977385,2.358598,4.705371
48,4.898689,3.569910,2.786384,2.366790,2.127134,3.166534,4.166453,0.528417,4.627518,3.837013,...,0.382473,1.930240,2.918337,0.123445,0.747106,1.974510,0.562224,0.498717,4.202867,3.018211
37,4.097525,2.892459,1.690617,1.443977,3.338252,2.448443,3.279863,0.403653,4.035614,4.442106,...,1.336291,4.626662,4.741264,3.824411,2.090604,4.124967,3.249689,4.857107,0.875680,0.338901


In [144]:
# za pomoca sprytnej sztuczki przeksztalcamy oceny z formatu dostarczonego przez MovieLens do uzytecznej macierzy
# zwroc uwage na to, ze czesci filmow moze brakowac po podziale datasetu na dwie czesci - musimy uzueplnic brakujace kolumny

def get_ratings(raw_ratings, movies):
    ratings = raw_ratings.pivot(*raw_ratings.columns).fillna(0.0)
    missing_movies = set(movies.index).difference(set(raw_ratings['movieId']))
    for movie in missing_movies:
        ratings[movie] = 0.0
    ratings = ratings.reindex(sorted(ratings.columns), axis=1)
    return ratings

ratings = get_ratings(train_ratings_set, movies)
ratings

/tmp/ipykernel_17369/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Część 2. - trening modelu

In [145]:
# trenujemy model iteracyjnie, wykorzystujac gradient descent

alpha = 0.0001 # learning speed
delta = 100 # minimal upgrade for each step
lambd = 0.01 # regularization weight

def calculate_user_preferences(users, movies, ratings, raw_ratings, users_no, movies_no, alpha, delta, lambd):
    total_error = 0.0
    model = users

    while(True):
        previous_total_error = total_error
        movies_t = movies.transpose()

        predicted_ratings = model.dot(movies_t) # mozemy to policzyc jako iloczyn skalarny preferencji uzytkownikow i cech filmow
        # tu stosujemy bardzo przydatna funkcje NumPy
        errors = np.where(ratings==0.0, pandas.DataFrame(np.zeros((users_no, movies_no))), predicted_ratings - ratings)
        gradient = errors.dot(movies) # znow iloczyn skalarny - tym razem bledow

        # tu stosujemy pewna sztuczke - rozbijamy sobie macierz z wyrazami regularyzujacymi na dwie
        # pierwsza to kolumna zlozona z zer
        regularization_k0 = pandas.DataFrame(np.zeros((users_no, 1)), index=raw_ratings['userId'].unique(), columns=['bias'])
        # druga to macierz preferencji uzytkownikow (czyli modelu) - bez pierwszej kolumny
        regularization_k = model.drop(columns=['bias'])
        # teraz sklejamy obie macierze
        regularization = pandas.concat([regularization_k0, regularization_k], axis=1)

        # najwazniejszy krok - aktualizacja modelu, czyli wszystkich wag
        model = model - alpha * (gradient + lambd * regularization)

        total_error = np.sum(errors ** 2)
        print(total_error)
        progress = abs(previous_total_error - total_error)
        if progress < delta:
            break
        if (abs(previous_total_error) < abs(total_error)) and previous_total_error != 0:
            break
            
    return model

prediction_model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)

4802810.397778639
4606991.180263925
4436986.3557882095
4291671.87611832
4170006.9456234328
4071029.5014521508
3993852.0202630614
3937657.6313303704
3901696.5180140054
3885282.591593228
3887790.423323959


## Część 3. - ocena jakości algorytmu

In [146]:
# na podstawie zbioru testowego i wytrenowanego modelu obliczamy metryki opisujace jakosc modelu

positive_threshold = 4.0
negative_threshold = 2.0

def calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold):
    # obliczamy true_positives itp.
    test_ratings_set = test_ratings_set.set_index(['userId', 'movieId'])
    predicted_ratings.index.name = 'userId'
    predicted_ratings = predicted_ratings.unstack().reset_index(name='rating').set_index(['userId', 'movieId'])
    
    test_ratings = test_ratings_set.values.reshape(-1)
    predicted_ratings = predicted_ratings.loc[test_ratings_set.index].values.reshape(-1)

    def thresholds(x):
        return True if x <= negative_threshold or x >= positive_threshold else False

    def normalize(x):
        if x <= negative_threshold:
            return 0
        if x >= positive_threshold:
            return 1

    ratings = [(t,p) for (t,p) in  zip(test_ratings,predicted_ratings) if thresholds(t) and thresholds(p)]        

    test_ratings = [normalize(x) for (x,_) in ratings]
    predicted_ratings = [normalize(x) for (_,x) in ratings]

    true_negatives, false_positives, false_negatives, true_positives = sklearn.metrics.confusion_matrix(test_ratings, predicted_ratings, labels=[0, 1]).ravel()
    # nastepnie wszystkie metryki
    accuracy = (true_negatives + true_positives) / (true_negatives + false_positives + false_negatives + true_positives)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1 = 2*precision*recall / (precision + recall)
        
    return {
        'true_positives': true_positives,
        'true_negatives': true_negatives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [155]:
predicted_ratings = prediction_model.dot(movies.T)
calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold)

{'true_positives': 838,
 'true_negatives': 95,
 'false_positives': 84,
 'false_negatives': 134,
 'accuracy': 0.8105994787141616,
 'precision': 0.9088937093275488,
 'recall': 0.8621399176954733,
 'f1': 0.8848996832101372}

In [148]:
# dla porownania - obliczmy te same metryki dla modelu losowego
# zauwaz, w jaki sposob ponownie wykorzystujemy funkcje inicjalizujaca preferencje uzytkownikow

_, random_model = initialize_users(train_ratings_set)
random_prediction = random_model.dot(movies.T)
calculate_stats(test_ratings_set, random_prediction, positive_threshold, negative_threshold)

{'true_positives': 2209,
 'true_negatives': 20,
 'false_positives': 599,
 'false_negatives': 52,
 'accuracy': 0.7739583333333333,
 'precision': 0.7866809116809117,
 'recall': 0.9770013268465281,
 'f1': 0.8715723022292365}

## Część 4. - istotność statystyczna

In [150]:
# wielokrotnie uruchamiamy trening modelu
# za każdym razem dzielimy dataset na zbior treningowy i testowy w inny sposob - klasa KFold robi to za nas
# zwroc uwage na bardzo istotny szczegol - oba modele, wytrenowany i losowy, musza byc porownywane na tym samym zbiorze testowym

n_tests = 5
results = []
random_results = []

for train, test in KFold(n_splits=n_tests, shuffle=True).split(all_ratings):# wygeneruj macierz użytkowników i ocen
    users_no, users = initialize_users(all_ratings)
    train_ratings = all_ratings.iloc[train]
    ratings = get_ratings(train_ratings, movies)
    # wytrenuj model
    prediction_model = calculate_user_preferences(users, movies, ratings, train_ratings, users_no, movies_no, alpha, delta, lambd)
    # oblicz metryki dla wytrenowanego modelu
    test_ratings = all_ratings.iloc[test]
    predicted_ratings = prediction_model.dot(movies.T)
    results.append(calculate_stats(test_ratings, predicted_ratings, positive_threshold, negative_threshold))
    # oblicz metryki dla modelu losowego
    _, random_model = initialize_users(train_ratings)
    random_prediction = random_model.dot(movies.T)
    random_results.append(calculate_stats(test_ratings, random_prediction, positive_threshold, negative_threshold))

/tmp/ipykernel_17369/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


4176871.2552945307
3628934.041169085
3245444.1695792982
2958753.5183542604
2733089.825172172
2548529.8440238065
2393317.9075973267
2260090.973489007
2143966.2643432994
2041536.8130496135
1950321.4284189574
1868449.3493354297
1794470.581325147
1727236.320134983
1665820.184690446
1609464.2759944878
1557540.9891241558
1509525.2157118637
1464973.6379476883
1423509.0064989582
1384808.0088921904
1348591.7788708329
1314618.3827581177
1282676.8081518267
1252582.1091806206
1224171.4523762148
1197300.8710571253
1171842.5822698867
1147682.7541869425
1124719.6370135506
1102861.989358668
1082027.7463759142
1062142.8869784523
1043140.465932309
1024959.7832559801
1007545.6685536125
990847.862019646
974820.4771242635
959421.5326091973
944612.5435345588
930358.1628280111
916625.8661812688
903385.6742800836
890609.9072928494
878272.9673191682
866351.1451441272
854822.4481812004
843666.4469363944
832864.1377038169
822397.8195214355
812250.9836849844
802408.2143466444
792855.0989196491
783578.1471760753
7

/tmp/ipykernel_17369/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


4001164.6996260947
3548187.2876699064
3206781.5894296477
2938745.799816818
2721452.355509811
2540837.7973673353
2387741.997995256
2255933.1307816147
2141004.0261019287
2039732.1668431414
1949694.1394235308
1869024.3944546909
1796258.6893085414
1730228.973048369
1669990.6082930814
1614770.5976810842
1563929.8771068
1516935.294321378
1473338.4225548585
1432759.3028423819
1394873.8075714284
1359403.7080592478
1326108.7898442843
1294780.5378096383
1265237.0378535346
1237318.8304377447
1210885.5153983044
1185812.9543671426
1161990.9520223723
1139321.323564788
1117716.2756704246
1097097.0433591844
1077392.736937781
1058539.3622830398
1040478.9848599213
1023159.0134836381
1006531.5842851298
990553.0288857444
975183.4136288912
960386.1390053235
946127.5902610113
932376.8316827734
919105.3382866529
906286.7596427914
893896.7114010187
881912.5907680497
870313.4127567313
859079.6645022552
848193.1753365105
837637.0006442324
827395.3178040257
817453.3327532363
807797.1959153208
798413.9263977015
7

/tmp/ipykernel_17369/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


4081481.849862146
3638225.063128881
3298070.994619132
3025998.581600918
2801912.711719793
2613409.2748750485
2452299.6382731176
2312866.35318718
2190939.2828717097
2083375.4784672162
1987747.360587047
1902144.460622257
1825040.8338119404
1755202.7057302126
1691622.0763496659
1633467.811332938
1580048.9271206802
1530786.6098547457
1485192.6219672887
1442852.4585825144
1403412.084012518
1366567.3977818126
1332055.8026067324
1299649.4056134985
1269149.4990283833
1240382.0508308294
1213193.9983035733
1187450.1841398375
1163030.8100383389
1139829.3095550863
1117750.5625553082
1096709.389493279
1076629.2760928213
1057441.2886569665
1039083.1478331655
1021498.4346736976
1004635.9076165729
988448.912840868
972894.8735291956
957934.846057607
943533.1331527202
929656.945702491
916276.1062555735
903362.788352971
890891.2867510599
878837.8143535684
867180.322300879
855898.340191525
844972.8338515516
834386.0784378429
824121.5449735271
814163.798677424
804498.4076731174
795111.8608532737
785991.493

/tmp/ipykernel_17369/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


4029029.2414342538
3539981.803431755
3183511.1133636143
2909961.0077100275
2691524.1849122522
2511754.5226846943
2360374.2034764593
2230616.681841525
2117815.065785277
2018620.342007386
1930548.9541334417
1851709.0921435968
1780627.8602612058
1716137.7449597355
1657299.3625270035
1603347.2585830486
1553650.8715643452
1507685.784697713
1465012.1502379428
1425258.2324039054
1388107.678282809
1353289.5519019414
1320570.4478626968
1289748.1911240222
1260646.760960571
1233112.169703302
1207009.0931961478
1182218.098139236
1158633.3470473215
1136160.6880842275
1114716.0570512228
1094224.134060679
1074617.2091574601
1055834.2202483846
1037819.9338108439
1020524.244449163
1003901.5738025322
987910.3528443679
972512.574448045
957673.4053790555
943360.8487246144
929545.4492778997
916200.0356251551
903299.4936939835
890820.5673533552
878741.6823439138
867042.7903880349
855705.2308042524
844711.6073476964
834045.6783304925
823692.258355532
813637.1302323311
803866.9658425862
794369.2548915127
7851

/tmp/ipykernel_17369/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


4316462.609796415
3737695.0746036237
3327410.389257848
3018825.782136691
2776005.3829650925
2578396.1185975727
2413485.57686386
2273193.612727643
2152024.4275408667
2046074.7893139771
1952474.163963809
1869052.058781716
1794130.4721621855
1726388.5532173982
1664770.908764547
1608423.451622789
1556647.3069735905
1508864.9581929285
1464594.9248176657
1423432.5301456102
1385035.101935345
1349110.4545215792
1315407.8345941135
1283710.7394876084
1253831.1740280485
1225605.0231298178
1198888.2972256397
1173554.065854614
1149489.9377272397
1126595.9776725355
1104782.9750526715
1083970.9966131856
1064088.1708229007
1045069.6616303731
1026856.7980120647
1009396.3322936217
992639.8054210348
976543.001467486
961065.4769281514
946170.1529650745
931822.9608594766
917992.532618554
904649.9300527046
891768.4067526828
879323.19830604
867291.3368383251
855651.4865790894
844383.7976606549
833469.7757790087
822892.1656971548
812634.8468644656
802682.7396714664
793021.7210663719
783638.5484343087
774520.7

In [151]:
# obliczamy, w ilu probach wytrenowany model okazal sie lepszy od losowego
# przeprowadzamy test statystyczny - jak prawdopodobne jest to, by k pozytywnych prob bylo dzielem przypadku

def possibility_of_at_least_k_successes_in_n(k, n):
    p = 0.0
    # obliczamy kolejno prawdopodobienstwo k sukcesow, k+1 sukcesow, ...
    # przydadza Ci sie funkcje marh.comb() i math.pow()
    for i in range(k, n+1):
        p += comb(n, i) * math.pow(0.5, n)
    return p

p = 0.05
metric = 'recall'

positive_tests_count = len(list(filter(lambda results: results[0][metric] > results[1][metric],zip(results, random_results))))# w ilu przypadkach okazalismy sie lepsi niz random?
print(positive_tests_count)
if possibility_of_at_least_k_successes_in_n(positive_tests_count, n_tests) <= p:
    print('We are better than random!')
else:
    print('There is no evidence we are better')

0
There is no evidence we are better
